# Descriptors
Image Patch -- Just use the pixel values of the patch
- Perfectly fine if geometry and appearance is unchanged
- A downsampling is useful to make it simple, fast, robust to small affine transforms.

Image Gradients
- Feature is invariant to absolute intensity values
- Sensitive to deformations

Color histogram
- Invariant to changes in scale and rotation
- Sensitive to spatial layout

Spatial histograms
- Retains rough spatial layout
- Some invariance to deformations
- Orientation normalization may useful -- Use the dominant image gradient direction to normalize the orientation of the patch

Multi-Scale Oriented Patches 